In [1]:
import torch
from torch import optim, nn
from torch.autograd import Variable
import syft as sy
hook = sy.TorchHook(torch)
# import pixiedust

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/adamhall/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0.so'


In [2]:
@property
def location(self):
    m = self.__getitem__(0)
    w = m.weight[0]
    return w.location

nn.Sequential.location = location

In [3]:
# A Toy Dataset
x = torch.tensor([[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0],[1,1,0,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],[0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1.]])
x.requires_grad_()
target = torch.tensor([[0],[0],[0],[0],[0],[0],[0],[0],[1],[1],[1],[1],[1],[1],[1],[1.]])


#   Variables for performance metrics
epochs = 20
lr = 0.2
counter = 0

# Define 2 chained models
models = [
    nn.Sequential(
        nn.Linear(4, 3),
        nn.Tanh()
    ),
    nn.Sequential(
        nn.Linear(3, 1),
        nn.Sigmoid()
    )
]

# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(params=model.parameters(),lr=lr)
    for model in models
]

# create some workers
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
workers = alice, bob

# Send Model Segments and Data to starting locations
model_locations = [alice, bob]

for model, location in zip(models, model_locations):
    model.send(location)
    
x = x.send(models[0].location)
target = target.send(models[1].location)

In [5]:
# %%pixie_debugger

def train():
    # Training Logic
    for iter in range(epochs):

        # 1) erase previous gradients (if they exist)
        for opt in optimizers:
            opt.zero_grad()

        # 2) make a prediction
        a  = models[0](x)
        
        # 3) send the activation signal to the next model
        a_to_send = a.detach()
        remote_a = a_to_send.move(models[1].location)
        # re-enable autograd here
        remote_a.requires_grad_()

        pred =  models[1](remote_a)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
#         print(remote_a.location._objects[remote_a.grad.id_at_location])  <-- throws error NoneType hs no location (Expected as no gradients here yet)
        loss.backward()
        
        
        # Now the gradient produced by backward seems to be a copy of the original tensor :S
        if remote_a.grad.location._objects[remote_a.grad.id_at_location].sum() == remote_a.location._objects[remote_a.id_at_location].sum():
            print("\n\nGradient of 'a' after backward() appears to just be a copy of 'a'")
            print(remote_a.grad.location._objects[remote_a.grad.id_at_location])
            print(remote_a.location._objects[remote_a.id_at_location])
        
        # 5) Backprop gradient to model behind
        grad_a = remote_a.grad
        grad_a.move(models[0].location)
        a.backward(grad_a)


        # 5) change the weights
        for opt in optimizers:
            opt.step()

        # 6) print our progress
        # Do not use .data
        print(loss.detach().get())
        
train()



Gradient of 'a' after backward() appears to just be a copy of 'a'
tensor([[-0.5728,  0.3337,  0.6599],
        [-0.6832,  0.6359,  0.8446],
        [-0.6823,  0.5808,  0.8423],
        [-0.6931,  0.3845,  0.8503],
        [-0.7686,  0.7887,  0.9320],
        [-0.7768,  0.6694,  0.9356],
        [-0.7761,  0.6182,  0.9346],
        [-0.8394,  0.8098,  0.9726],
        [-0.6828,  0.5196,  0.8590],
        [-0.7689,  0.7531,  0.9395],
        [-0.7682,  0.7127,  0.9385],
        [-0.7765,  0.5610,  0.9418],
        [-0.8335,  0.8609,  0.9743],
        [-0.8396,  0.7773,  0.9757],
        [-0.8391,  0.7403,  0.9753],
        [-0.8857,  0.8753,  0.9898]], requires_grad=True)
tensor([[-0.5728,  0.3337,  0.6599],
        [-0.6832,  0.6359,  0.8446],
        [-0.6823,  0.5808,  0.8423],
        [-0.6931,  0.3845,  0.8503],
        [-0.7686,  0.7887,  0.9320],
        [-0.7768,  0.6694,  0.9356],
        [-0.7761,  0.6182,  0.9346],
        [-0.8394,  0.8098,  0.9726],
        [-0.6828,  0.51



Gradient of 'a' after backward() appears to just be a copy of 'a'
tensor([[-0.2874, -0.5044,  0.4245],
        [-0.5007, -0.6764,  0.8020],
        [-0.5005, -0.6780,  0.8013],
        [-0.5029, -0.6823,  0.8035],
        [-0.6666, -0.7979,  0.9417],
        [-0.6683, -0.8008,  0.9424],
        [-0.6682, -0.8018,  0.9422],
        [-0.7864, -0.8789,  0.9839],
        [-0.5006, -0.6795,  0.8057],
        [-0.6666, -0.7989,  0.9431],
        [-0.6665, -0.8000,  0.9429],
        [-0.6683, -0.8028,  0.9436],
        [-0.7853, -0.8777,  0.9841],
        [-0.7865, -0.8796,  0.9843],
        [-0.7864, -0.8802,  0.9843],
        [-0.8659, -0.9281,  0.9957]], requires_grad=True)
tensor([[-0.2874, -0.5044,  0.4245],
        [-0.5007, -0.6764,  0.8020],
        [-0.5005, -0.6780,  0.8013],
        [-0.5029, -0.6823,  0.8035],
        [-0.6666, -0.7979,  0.9417],
        [-0.6683, -0.8008,  0.9424],
        [-0.6682, -0.8018,  0.9422],
        [-0.7864, -0.8789,  0.9839],
        [-0.5006, -0.67